In [1]:
import pandas as pd
import numpy as np

# Clasificación de champiñones

Utilizaremos el dataset de champiñones obtenido [aquí](https://www.kaggle.com/uciml/mushroom-classification) para 
hacer un breve ejercicio de clasificación. El dataset consiste de 8124 muestras correspondientes a 23 diferentes 
especies de hongos, cada uno 22 predictores y una clasificación "venenoso" y "no venenoso". 

El dataset fue dividido en 70% entrenamiento, 21% validación y 9% prueba. Para generar nuestras predicciones utilizamos una red neuronal. La idea era mostrar cómo cambiaba el accuracy de nuestras predicciones modificando la arquitectura de la red (añadiendo capas y jugando con el número de neuronas en cada capa) sin embargo el modelo más sencillo posible (no capas ocultas, activación sigmoide), equivalente a la regresión logística, nos da un accuracy en el set de test del 100%.

In [2]:
df = pd.read_csv("mushrooms.csv") #cargamos el dataset con pandas

In [3]:
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [4]:
df.describe() 

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


```class``` es la variable que queremos predecir, el resto de las categorías funcionarán como nuestros predictores. 
TODO: Hacer feature selection, para hacer el modelo lo más parsimonioso posible sin sacrificar mucho accuracy


In [5]:
from sklearn.preprocessing import OneHotEncoder #importamos algunas herramientas de preprocesamiento
from sklearn.model_selection import train_test_split

In [6]:
X = df.iloc[:,1:]
Y = df.iloc[:,0]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y) #dividimos el dataset 

In [8]:
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test) #dividimos el set de prueba en test y validación

In [9]:
ohe_X = OneHotEncoder() #Transformamos nuestro dataset, consistente de variables categóricas, a numéricas
X_train_ohe = ohe_X.fit_transform(X_train).toarray()
print(X_train_ohe.shape)

(6093, 117)


In [10]:
X_train_ohe

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [11]:
X_test_ohe = ohe_X.transform(X_test).toarray() #transformamos el dataset de prueba

In [13]:
X_val_ohe = ohe_X.transform(X_val).toarray() #transformamos el dataset de val.
print(X_val_ohe.shape) #tenemos 508 ejemplos de validación, cada ejemplo tiene 117 features (resultantes del ohe)

(508, 117)


In [14]:
Y_train = Y_train.to_numpy().reshape(-1,1)
Y_val = Y_val.to_numpy().reshape(-1,1)
Y_test = Y_test.to_numpy().reshape(-1,1)

In [15]:
ohe_label = OneHotEncoder() #finalmente transformamos los labels a numerico
Y_train_ohe = ohe_label.fit_transform(Y_train).toarray()
Y_train_ohe = Y_train_ohe[:,1].reshape(-1,1)
print(Y_train_ohe.shape) #usaremos 6093 ejemplos para entrenar el modelo

(6093, 1)


In [16]:
Y_val_ohe = ohe_label.transform(Y_val).toarray()
Y_val_ohe = Y_val_ohe[:,1].reshape(-1,1)
print(Y_val_ohe.shape)

(508, 1)


In [17]:
Y_test_ohe = ohe_label.transform(Y_test).toarray()
Y_test_ohe = Y_test_ohe[:,1].reshape(-1,1)

In [19]:
Y_train_ohe

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]])

In [20]:
X_val_ohe.shape, Y_val_ohe.shape

((508, 117), (508, 1))

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [38]:
#Definimos el modelo, en este caso una sola capa fully connected con activación sigmoide
model = keras.Sequential(
    [
        layers.Dense(1, activation = "sigmoid", name = "output")
    ]
)

In [39]:
model(X_train_ohe)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(6093, 1), dtype=float32, numpy=
array([[0.41541094],
       [0.38343585],
       [0.5293954 ],
       ...,
       [0.30215803],
       [0.4277146 ],
       [0.2956047 ]], dtype=float32)>

In [40]:
#usaremos entropia binaria cruzada como funcion de costo, optimizaremos con ADAM, y tendremos de métrica el accuracy
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = "accuracy")

In [41]:
history = model.fit(X_train_ohe, Y_train_ohe, batch_size=128,validation_data = (X_val_ohe, Y_val_ohe), epochs = 100)

Epoch 1/100
48/48 [==============================] - 0s 5ms/step - loss: 0.6151 - accuracy: 0.6709 - val_loss: 0.5386 - val_accuracy: 0.7894
Epoch 2/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4957 - accuracy: 0.8173 - val_loss: 0.4406 - val_accuracy: 0.8642
Epoch 3/100
48/48 [==============================] - 0s 2ms/step - loss: 0.4155 - accuracy: 0.8533 - val_loss: 0.3744 - val_accuracy: 0.8799
Epoch 4/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3596 - accuracy: 0.8671 - val_loss: 0.3271 - val_accuracy: 0.8937
Epoch 5/100
48/48 [==============================] - 0s 2ms/step - loss: 0.3179 - accuracy: 0.8779 - val_loss: 0.2916 - val_accuracy: 0.8957
Epoch 6/100
48/48 [==============================] - 0s 3ms/step - loss: 0.2853 - accuracy: 0.8861 - val_loss: 0.2636 - val_accuracy: 0.9035
Epoch 7/100
48/48 [==============================] - 0s 2ms/step - loss: 0.2586 - accuracy: 0.8945 - val_loss: 0.2407 - val_accuracy: 0.9075
Epoch 8/100
4

Epoch 59/100
48/48 [==============================] - 0s 3ms/step - loss: 0.0304 - accuracy: 0.9984 - val_loss: 0.0326 - val_accuracy: 0.9961
Epoch 60/100
48/48 [==============================] - 0s 2ms/step - loss: 0.0297 - accuracy: 0.9984 - val_loss: 0.0318 - val_accuracy: 0.9961
Epoch 61/100
48/48 [==============================] - 0s 3ms/step - loss: 0.0290 - accuracy: 0.9984 - val_loss: 0.0311 - val_accuracy: 0.9961
Epoch 62/100
48/48 [==============================] - 0s 3ms/step - loss: 0.0283 - accuracy: 0.9984 - val_loss: 0.0304 - val_accuracy: 0.9961
Epoch 63/100
48/48 [==============================] - 0s 3ms/step - loss: 0.0276 - accuracy: 0.9984 - val_loss: 0.0297 - val_accuracy: 0.9961
Epoch 64/100
48/48 [==============================] - 0s 2ms/step - loss: 0.0270 - accuracy: 0.9984 - val_loss: 0.0291 - val_accuracy: 0.9961
Epoch 65/100
48/48 [==============================] - 0s 2ms/step - loss: 0.0263 - accuracy: 0.9985 - val_loss: 0.0283 - val_accuracy: 0.9961
Epoch 

In [42]:
model.evaluate(X_test_ohe, Y_test_ohe) 
#Después de 100 épocas de entrenamiento tenemos un 100% de accuracy en el test set

48/48 [==============================] - 0s 969us/step - loss: 0.0111 - accuracy: 1.0000


[0.011074339970946312, 1.0]

TODO: Revisar la distribución de las clases para ver qué tan óptimo es el accuracy como métrica. Si las clases no están balanceadas (hay una diferencia considerable entre casos venenosos y no venenosos) entonces valdría la pena añadir la tasa de falsos positivos y falsos negativos para monitorear el entrenamiento del modelo. 
